In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

In [ ]:
df = pd.read_csv("working_data/mhs_sleep_ch_without_outliers.csv")
df['LOWER_DAYS'] = pd.to_datetime(df['LOWER_DAYS'])
df['WEEKDAY'] = df['LOWER_DAYS'].dt.day_name()
print(df.head())

In [ ]:
print(df.iloc[0])

In [ ]:
# helper function to find first monday
def filter_from_first_monday(group):
    monday_idx = group.index[group['LOWER_DAYS'].dt.weekday == 0]
    if monday_idx.empty:
        return pd.DataFrame()
    start_idx = monday_idx.min()
    return group.loc[start_idx:]

df = df.groupby('USER_ID', group_keys=False).apply(filter_from_first_monday)

print(df.columns)

df['WEEKDAY_STR'] = df['LOWER_DAYS'].dt.day_name().str[:3] # add weekday string to check later
df['WEEKDAY_NUM'] = df['LOWER_DAYS'].dt.weekday # add weekday number from 0=Monday to 6=Sunday

df['WEEK_START'] = df['LOWER_DAYS'] - pd.to_timedelta(df['LOWER_DAYS'].dt.weekday, unit='D')

print(df.columns)

# find weeks with values for every day
def check_continuous_dates(group):
    expected_days = set(range(7))
    actual_days = set(group['WEEKDAY_NUM'].unique())

    # drop week if day is missing
    if actual_days != expected_days:
        return pd.DataFrame()
    else:
        return group

# check for continuous dates
df = df.groupby(['USER_ID', 'WEEK_START']).apply(check_continuous_dates)

print(df.columns)

for i, day in enumerate(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']):
    df[f'DATE_{day}'] = df['WEEK_START'] + pd.to_timedelta(i, unit='D')
    
print(df.columns)

id_vars = ['USER_ID', 'WEEK_START', 'WEEKDAY_STR']
value_vars = [col for col in df.columns if col not in id_vars + ['LOWER_DAYS', 'WEEKDAY_NUM', 'DATE_Mon', 'DATE_Tue', 'DATE_Wed', 'DATE_Thu', 'DATE_Fri', 'DATE_Sat', 'DATE_Sun']]

melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='feature', value_name='value')

melted['feature_day'] = melted['feature'] + '_' + melted['WEEKDAY_STR']

weekly_df = melted.pivot_table(index=['USER_ID', 'WEEK_START'], columns='feature_day', values='value', aggfunc='first')

weekly_df.columns.name = None
weekly_df = weekly_df.reset_index()

for i, day in enumerate(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']):
    weekly_df[f'DATE_{day}'] = pd.to_datetime(weekly_df['WEEK_START']) + pd.to_timedelta(i, unit='D')

print(weekly_df.head())

In [ ]:
# sort all parameters according to weekdays

weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

weekday_columns = [col for col in weekly_df.columns if any(col.endswith(f"_{day}") for day in weekdays)]

ordered_columns = ['USER_ID', 'WEEK_START'] + sorted(weekday_columns, key=lambda x: weekdays.index(x.split('_')[-1]))

weekly_df = weekly_df[ordered_columns]

print(weekly_df.head())

In [ ]:
pd.set_option('display.max_columns', None)

print(weekly_df.columns.tolist()[:33])

In [ ]:
# check whether there are still NaN values or whether they have been sorted out

print("Number of rows:", len(weekly_df))
print("Number of rows with NaN values:", weekly_df.isna().any(axis=1).sum())

In [ ]:
weekly_df.to_csv("working_data/mhs_sleep_weekly.csv", index=False)